In [1]:
import xarray as xr
import numpy as np
import pyproj
import s3fs


ds_cfgrib = xr.open_dataset('../../tests/data/hrrr.t06z.wrfsfcf01-TMP.grib2', engine='cfgrib')
ds_cfgrib

<xarray.Dataset>
Dimensions:     (y: 1059, x: 1799)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
    latitude    (y, x) float64 ...
    longitude   (y, x) float64 ...
    valid_time  datetime64[ns] ...
Dimensions without coordinates: y, x
Data variables:
    t           (y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-05-06T10:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [2]:
ds_gribberish = xr.open_dataset('../../tests/data/hrrr.t06z.wrfsfcf01-TMP.grib2', engine='gribberish')
ds_gribberish

/Users/matthew/Developer/gribberish/python/gribberish/gribberish_backend.py:51: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  return xr.Dataset(


<xarray.Dataset>
Dimensions:    (time: 1, y: 1059, x: 1799)
Coordinates:
  * time       (time) datetime64[ns] 2023-07-22T07:00:00
  * y          (y) float64 -1.587e+06 -1.584e+06 ... 1.584e+06 1.587e+06
  * x          (x) float64 -2.698e+06 -2.695e+06 ... 2.693e+06 2.696e+06
    latitude   (y, x) float64 ...
    longitude  (y, x) float64 ...
Data variables:
    tmp        (time, y, x) float64 ...
Attributes:
    meta:     Generated with gribberishpy

Verify that the coordinates match between cfgrib and gribberish

In [3]:
lat_cfgrib = ds_cfgrib.latitude.values
lon_cfgrib = ds_cfgrib.longitude.values
lat_gribberish = ds_gribberish.latitude.values
lon_gribberish = ds_gribberish.longitude.values

np.allclose(lat_cfgrib, lat_gribberish), np.allclose(lon_cfgrib, lon_gribberish + 360.0)

(True, True)

In [5]:
to_latlon = pyproj.Transformer.from_proj(ds_gribberish.tmp.crs, 'epsg:4326', always_xy=True).transform

xx, yy = np.meshgrid(ds_gribberish.x.values, ds_gribberish.y.values)

lons, lats = to_latlon(xx, yy)

In [7]:
np.allclose(lats, lat_cfgrib), np.allclose(lons + 360, lon_cfgrib)

(True, True)